In [18]:
## Goals of this experiment:
## - SGHMCHD is more robust with respect to different initial (smaller) stepsizes than SGHMC [x]
## - SGHMCHD may outperform SGHMC with best stepsize over coarse grid in certain cases, even when initialized
## badly (and will perform comparably most of the time.) [x]
%matplotlib inline
import numpy as np
import sys
from os.path import dirname, join as path_join, isdir
from glob import glob
import json
import pandas as pd
import re

from collections import defaultdict

sys.path.insert(0, path_join("..", "..", ".."))
from pysgmcmc_experiments.utils import (
    rename, format_tex as format_stepsize,
    format_performance,
    format_sampler
)

RESULT_DIR = path_join("..", "..", "..", "cluster_results", "uci")


def load_results(directory):
    directories = [
        directory for directory in glob("{}/*".format(directory))
        if isdir(directory)
    ]
    
    def load_json(filename):
        with open(filename) as f:
            return json.load(f)
    
    configurations = tuple(
        load_json(path_join(directory, "config.json"))
        for directory in directories
        if "_sources" not in directory
    )
    
    results = tuple(
        load_json(path_join(directory, "run.json"))
        for directory in directories
        if "_sources" not in directory
    )
    
    
    # runs = defaultdict(lambda: defaultdict(list))
    runs = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    for configuration, result in zip(configurations, results):
        if result["status"] == "COMPLETED":
            sampler = configuration["sampler"]
            stepsize = configuration["stepsize"]
            
            for dataset, dataset_result in result["result"].items():
                dataset_result["dataset"] = dataset
                runs[dataset][stepsize][sampler].append(dataset_result)
            
    return runs

runs = load_results(RESULT_DIR)

def results_for(metric_function, is_loss=True):
    performances = defaultdict(lambda: defaultdict(dict))
    for dataset, dataset_results in runs.items():
        for stepsize, stepsize_results in dataset_results.items():
            for sampler, sampler_runs in stepsize_results.items():
                performances_ = np.asarray([
                    metric_function(y_true=run["y_test"],
                                    prediction_mean=run["prediction_mean"],
                                    prediction_variance=run["prediction_variance"])
                    for run in sampler_runs
                ])
                mean_performance = performances_.mean()
                std_performance = performances_.std()
                performances[dataset][stepsize][sampler] = (mean_performance, std_performance)
    
    records = defaultdict(dict)

    if metric_function.__name__ == "log_likelihood":
        records["BostonHousing"] = {
            "VI": "$-2.903 \\pm 0.071$",
            "PBP": "$-2.574 \\pm 0.089$",
        }
        records["YachtHydrodynamics"] = {
            "VI": "$-3.439 \\pm 0.163$",
            "PBP": "$-1.634 \\pm 0.016$"
        }
        records["Concrete"] = {
            "VI": "$-3.391 \\pm 0.017$",
            "PBP": "$-3.161 \\pm 0.019$"
        }
        records["Wine Quality Red"] = {
            "VI": "$-0.980 \\pm 0.013$",
            "PBP": "$-0.968 \\pm 0.014$"
        }
    elif metric_function.__name__ == "rmse":
        records["Boston Housing"] = {
            "VI": "$4.320 \\pm 0.2914$",
            "PBP": "$3.014 \\pm 0.1800$",
        }
        records["Yacht Hydrodynamics"] = {
            "VI": "$6.887 \\pm 0.6749$",
            "PBP": "$1.015 \\pm 0.0542$"
        }
        records["Concrete"] = {
            "VI": "$7.128 \\pm 0.1230$",
            "PBP": "$5.667 \\pm 0.0933$"
        }
        
        records["Wine Quality Red"] = {
            "VI": "$0.646 \\pm 0.0081$",
            "PBP": "$0.635 \\pm 0.0079$"
        }    
    else:
        raise NotImplementedError(
            "We do not have any literature results for metric function {}".format(
            metric_function.__name__
        ))
    methods = set(("VI", "PBP"))

    for dataset, dataset_results in runs.items():
        for stepsize, stepsize_results in dataset_results.items():
            for sampler, sampler_runs in stepsize_results.items():
                print("DATASET:", dataset)
                relevant_performances = [
                    mean_performance for mean_performance, _ in
                    performances[dataset][stepsize].values()
                ]
                optimum = min if is_loss else max
                method = format_sampler(sampler, stepsize)
                mean_performance, std_performance = performances[dataset][stepsize][sampler]
                records[rename(dataset)][method] = format_performance(
                    mean=mean_performance,
                    stddev=std_performance,
                    bold=mean_performance == optimum(relevant_performances) # winner gets boldfaced
                )
                
                methods.add(method)
                
    def method_key(method):
        if method == "PBP":
            return -float("inf")
        elif method == "VI":
            return -1000000
        else:
            import re
            stepsize_exponent = float(re.search("\{\{(-[0-9]+)\}\}", method).group(1))
            return (0.00001 * method.startswith("SGHMCHD")) + stepsize_exponent
    
    columns = [rename(dataset) for dataset in DATASETS]
    dataframe = pd.DataFrame.from_records(
        records, columns=[rename(dataset) for dataset in DATASETS],
        index=sorted(methods, key=method_key) 
    )
    dataframe.metric_name = rename(metric_function.__name__)
    dataframe.short_metric_name = metric_function.__name__
    return dataframe

    

Compute dataframes with results for all metrics and write them to files as latex.

In [17]:
from sklearn.metrics import mean_squared_error
import string

# XXX: Insert line between "literature results" and our results
def prettyprint_results(dataframe):
    table = dataframe.to_latex(escape=False).replace("{}", "Method/Dataset")
   
    table = table.rstrip("\n")
    caption = """\\caption{{{metric} for regression benchmarks from the UCI repository. 
For comparison we include results for VI~(variational inference) and
PBP~(probabilistic backpropagation) for Bayesian neural network
training taken from Hernández-Lobato and Adams~\\cite{{uci-vi-pbp}}.
We report mean $\\pm$ stddev across $10$ runs.}}""".format(metric=dataframe.metric_name)
    return """
\\begin{{table}}[H]
{table}
{caption}
{label}
\\end{{table}}
""".format(table=table, caption=caption, label="\label{tab:" + dataframe.short_metric_name + "}")

def rmse(y_true, prediction_mean, **kwargs):
    return np.sqrt(
        mean_squared_error(y_true=y_true, y_pred=prediction_mean)
    )


def log_variance_prior(log_variance, mean=1e-6, variance=0.01):
    return np.mean(np.sum(
        -np.square(log_variance - np.log(mean)) /
        (2 * variance) - 0.5 * np.log(variance), axis=1
    ))



def log_likelihood(y_true, 
                   prediction_mean, prediction_variance,
                   epsilon=1e-7):
    n_datapoints, _ = batch_size, _ = np.shape(y_true)
    f_mean = np.reshape(prediction_mean, (-1, 1))
    f_log_var = np.reshape(prediction_variance, (-1, 1))
    f_var_inv = 1. / (np.exp(f_log_var) + epsilon)
    
    mean_squared_error = np.square(y_true - f_mean)
    
    log_likelihood = np.sum(
        np.sum(
            -mean_squared_error * (0.5 * f_var_inv) - 
            0.5 * f_log_var, 
            axis=1
        )
    )
    log_likelihood /= batch_size
    log_likelihood += log_variance_prior(f_log_var) / n_datapoints
    return log_likelihood

y_true = np.random.rand(10, 1)
y_pred = np.random.rand(10, 2)
log_likelihood(y_true, y_pred[:, 0], y_pred[:, 1])

from os.path import join as path_join

OUTPUT_DIRECTORY = "/home/moritz/thesis_repos/masterthesis_report/sghmchd/experiments/results/tables/uci"
# metrics all take "y_test, prediction_mean, prediction_variance
# as argument and return a scalar loss value.
metric_functions = (rmse,)

for metric_function in metric_functions:
    metric_results = results_for(metric_function, is_loss=metric_function.__name__== "rmse")
    print(metric_results)
    results_tablecode = prettyprint_results(metric_results)
    print(results_tablecode)
    output_filename = path_join(
            OUTPUT_DIRECTORY, "{metric}.tex".format(metric=metric_function.__name__)
        )
    with open(output_filename, "w") as f:
        
        f.write(results_tablecode)

NameError: name 'results_for' is not defined